<h1> Content-Based Recommender Systems</h1>

![Recommendation Systems Approaches](https://raw.githubusercontent.com/ziababar/recommender/master/images/Recommendation%20Systems.jpg)

In content-based recommender systems, we use the content information of both users and items while building recommendation engines.

![Content Based Recommender Steps](https://raw.githubusercontent.com/ziababar/recommender/master/images/content-based-1.png)
A typical content-based recommender system will perform the following steps:
1. Retrieve user, item and activity data
2. Generate item profiles
3. Generate user profiles
4. Generate the recommendation engine model
5. Suggest the top N recommendations
![Content Based Recommender Steps](https://raw.githubusercontent.com/ziababar/recommender/master/images/content-based-2.png)

<h2>Step 1 - Retrieve Data</h2>
The first step would always be to gather the data and pull it into the programming environment.

For our use case, we download the MovieLens dataset containing three sets of data,
 - Movie data containing a certain movie's information, such as movieID, release date, URL, genre details, and so on
 - User data containing the user information, such as userID, age, gender, occupation, ZIP code, and so on
 - Ratings data containing userID, itemID, rating, timestamp

In [ ]:
# Import the libraries that are going to be used here
import pandas as pd
import numpy as np
import scipy
import sklearn

In [ ]:
# Column headers for the dataset
data_cols = ['user id','movie id','rating','timestamp']
item_cols = ['movie id','movie title','release date', 'video release date','IMDb URL','unknown','Action', 'Adventure','Animation','Childrens','Comedy','Crime', 'Documentary','Drama','Fantasy','Film-Noir','Horror', 'Musical','Mystery','Romance ','Sci-Fi','Thriller', 'War' ,'Western']
user_cols = ['user id','age','gender','occupation', 'zip code']

In [ ]:
# List of users
df_u_user = pd.read_csv('/home/nbuser/library/dataset/u.user', header=None, sep='|', names=user_cols, encoding='latin-1')
df_u_user = df_u_user.sort_values('user id', ascending=1)
df_u_user.columns
df_u_user.head(10)

In [ ]:
# List of movie items
df_u_item = pd.read_csv('/home/nbuser/library/dataset/u.item', header=None, sep='|', names=item_cols, encoding='latin-1')
df_u_item = df_u_item.sort_values('movie id', ascending=1)
df_u_item.columns
df_u_item.head(10)

In [ ]:
# Remove the several columns from the dataframe as it's not required for this analysis
df_u_item_final = df_u_item.drop(['release date', 'video release date','IMDb URL'], axis=1)
df_u_item_final.head(10)

In [ ]:
# User activity data
df_u_data = pd.read_csv('/home/nbuser/library/dataset/u.data', header=None, sep='\t', names=data_cols, encoding='latin-1')
df_u_data = df_u_data.sort_values('user id', ascending=1)
df_u_data.columns
df_u_data.head(10)

In [ ]:
# Remove the last two columns (rating and timestamp) from the dataframe as it's not required for this analysis.
# The specific rating value doesn't matter for this simplistic case, as long as the user has rated a movie.
df_u_data_final = df_u_data.drop(['timestamp', 'rating'], axis=1)
df_u_data_final.insert(2, 'rating', '1')
df_u_data_final.head(10)

<h2>Step 2 - Generate Item Profiles</h2>

In this step, we create a profile for each item using the content information we have about the items. The item profile is usually created using a widely-used information retrieval technique called TF-IDF. In information retrieval, TF-IDF (term frequency–inverse document frequency) is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus (Wikipedia).

Create the item profile using tf-idf functions available in the sklearn package. To generate tf-idf, we use the TfidfVectorizer(). The fit_transform() methods are in the sklearn package. The following code shows how we can create tfidf.


In [ ]:
# In the following code, the choice of the number of features to be included depends on the dataset,
# and the optimal number of features can be selected by the cross-validation approach:

from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(stop_words ="english", max_features = 100, ngram_range=(0,3), sublinear_tf =True)
x = v.fit_transform(df_u_item_final['movie title'])
profile_item = x.todense()

In [ ]:
profile_item

<h2>Step 3 - Generate User Profiles</h2>

In this step, we take the user activity dataset and preprocess the data into a proper format to create a user profile. We should remember that, in a content-based recommender system, the user profile is created with respect to the item content, that is, we have to extract or compute the preferences of the user for the item content or item features. Usually, a dot product between user activity and item profile gives us the user profile.

<h3> User Activity </h3>

A user-item rating density matrix needs to be created that where,
 - rows are the users IDs
 - columns are the item IDs
 - cells are the rating value (1 indicates if the user has ranked the web page, else 0) 

In [ ]:
# We use pivot to create binary rating matrix
rating_matrix = df_u_data_final.pivot(index='user id', columns='movie id', values='rating').fillna(0)

In [ ]:
rating_matrix.head(10)

A sparse matrix or sparse array is a matrix in which most of the elements are zero. By contrast, if most of the elements are nonzero, then the matrix is considered dense. The number of zero-valued elements divided by the total number of elements is called the sparsity of the matrix (which is equal to 1 minus the density of the matrix). Functionally, their behavior should be nearly identical to their dense counterparts.

The matrix above is a sparse matrix, as it has lots of zero values. Convert the sparse matrix to a dense matrix.

In [ ]:
# Create a dense matrix:
rating_matrix = rating_matrix.to_dense().as_matrix()
rating_matrix = rating_matrix.astype(float)
rating_matrix

<h3>User Profile</h3>

We now have item profile and user activity in hand; the dot product between these two matrices will create a new matrix with dimensions equal to # of users by # Item features.


In [ ]:
# Use the scipy package methods to compute the dot product between user activity and item profile
from scipy import linalg, dot
profile_user = dot(rating_matrix, profile_item) / linalg.norm(rating_matrix) / linalg.norm(profile_item)

In [ ]:
profile_user

<h2>Step 4 - Building Recommender Engine Model</h2>

Proceed to build a recommendation model now that the user profile and item profile is available.

Computing a cosine similarity between the user profile and item profile gives us the affinity of the user to each of the items. Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. 

![Cosine Similarity](https://www.safaribooksonline.com/library/view/statistics-for-machine/9781788295758/assets/2b4a7a82-ad4c-4b2a-b808-e423a334de6f.png "Cosine Similarity")

In [ ]:
# To compute the cosine calculations, we will be using the sklearn package.
# The following code will calculate the cosine similarity between userprofile an item profile

import sklearn.metrics
similarityCalc = sklearn.metrics.pairwise.cosine_similarity(profile_user, profile_item, dense_output=True)

In [ ]:
# We can see the results of the preceding calculation as follows:
similarityCalc

In [ ]:
# Now, let's format the preceding results calculated as binary data (0,1), as follows:

# First, we convert the rating to binary format:
final_pred = np.where(similarityCalc > 0.80, 1, 0)

In [ ]:
# Then we examine the final predictions of first three users:
final_pred[1]

Removing the zero values from the preceding results gives us the list of the probable items that can be recommended to the users:



In [ ]:
# For user 213 the recommended items are generated as follows:
indexes_of_user = np.where(final_pred[213] == 1)
indexes_of_user